# Resilient Financial Services AI Assistant

This notebook implements a comprehensive financial services AI assistant on Amazon Bedrock with:
- Foundation model benchmarking across Claude and Titan models
- Dynamic model routing with AWS AppConfig
- Circuit breaker patterns with Step Functions
- Cross-region deployment for high availability
- Model lifecycle management with SageMaker

**Architecture Components:**
1. Model evaluation and selection strategy
2. Flexible model abstraction layer
3. Resilient system design with fallbacks
4. Automated model customization and testing

## Section 1: Initialize AWS Clients and Configuration

Set up the AWS environment, import required libraries, and initialize clients for all services used in this implementation.

In [3]:
import boto3
import json
import time
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
import os
from typing import Dict, List, Any


In [4]:

# Initialize AWS clients
bedrock_runtime = boto3.client('bedrock-runtime', region_name='us-east-1')
appconfig_client = boto3.client('appconfig', region_name='us-east-1')
lambda_client = boto3.client('lambda', region_name='us-east-1')
apigateway_client = boto3.client('apigateway', region_name='us-east-1')
sfn_client = boto3.client('stepfunctions', region_name='us-east-1')
cloudformation_client = boto3.client('cloudformation', region_name='us-east-1')
route53_client = boto3.client('route53')
sagemaker_client = boto3.client('sagemaker', region_name='us-east-1')
s3_client = boto3.client('s3', region_name='us-east-1')
cloudwatch_client = boto3.client('cloudwatch', region_name='us-east-1')
iam_client = boto3.client('iam', region_name='us-east-1')

# Configuration variables
PRIMARY_REGION = 'us-east-1'
SECONDARY_REGION = 'us-west-2'
ENVIRONMENT = 'prod'
APP_NAME = 'AIAssistantApp'

# Bedrock model IDs to evaluate
MODELS_TO_EVALUATE = [
    "anthropic.claude-3-sonnet-20240229-v1:0",
    "anthropic.claude-instant-v1",
    "amazon.titan-text-express-v1"
]

# S3 bucket for storing datasets and models (replace with your bucket)
S3_BUCKET = f"cert-genai-dev/bonus_1_2/"

print("✓ AWS clients initialized successfully")
print(f"✓ Primary Region: {PRIMARY_REGION}")
print(f"✓ Secondary Region: {SECONDARY_REGION}")
print(f"✓ Environment: {ENVIRONMENT}")

✓ AWS clients initialized successfully
✓ Primary Region: us-east-1
✓ Secondary Region: us-west-2
✓ Environment: prod


## Part 1: Foundation Model Benchmarking

### Section 2: Setup Evaluation Framework

Implement the core evaluation infrastructure to benchmark Bedrock models on financial domain tasks.

In [3]:
def invoke_model(model_id: str, prompt: str, max_tokens: int = 500) -> Dict[str, Any]:
    """
    Invoke a Bedrock model with the given prompt and return response with metrics.
    
    Args:
        model_id: The Bedrock model identifier
        prompt: The input prompt to send to the model
        max_tokens: Maximum tokens for the response
        
    Returns:
        Dictionary containing success status, output, latency, and token count
    """
    start_time = time.time()
    
    try:
        # Prepare request body based on model provider
        if "anthropic" in model_id:
            body = json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": max_tokens,
                "messages": [
                    {"role": "user", "content": prompt}
                ],
                "temperature": 0.7,
                "top_p": 0.9
            })
        elif "amazon" in model_id:
            body = json.dumps({
                "inputText": prompt,
                "textGenerationConfig": {
                    "maxTokenCount": max_tokens,
                    "temperature": 0.7,
                    "topP": 0.9
                }
            })
        else:
            raise ValueError(f"Unsupported model provider in {model_id}")
        
        # Invoke the model
        response = bedrock_runtime.invoke_model(
            modelId=model_id,
            body=body
        )
        
        # Parse the response
        response_body = json.loads(response['body'].read().decode())
        
        # Extract output based on provider
        if "anthropic" in model_id:
            output = response_body['content'][0]['text']
            # Approximate token count for Claude
            token_count = len(output.split())
        elif "amazon" in model_id:
            output = response_body['results'][0]['outputText']
            # Get actual token count if available
            token_count = response_body['results'][0].get('tokenCount', len(output.split()))
        
        # Calculate metrics
        latency = time.time() - start_time
        
        return {
            "success": True,
            "output": output,
            "latency": latency,
            "token_count": token_count,
            "error": None
        }
        
    except Exception as e:
        return {
            "success": False,
            "output": None,
            "latency": time.time() - start_time,
            "token_count": 0,
            "error": str(e)
        }

print("✓ Model invocation function created")
print("✓ Supports Claude and Titan model families")

✓ Model invocation function created
✓ Supports Claude and Titan model families


### Section 3: Define Test Cases and Metrics

Create comprehensive financial domain test cases and implement similarity scoring.

In [4]:
# Financial domain test cases with ground truth answers
FINANCIAL_TEST_CASES = [
    {
        "question": "What is a 401(k) retirement plan?",
        "context": "Retirement planning and investment accounts",
        "ground_truth": "A 401(k) is a tax-advantaged retirement savings plan offered by employers that allows employees to contribute a portion of their salary before taxes are deducted.",
        "use_case": "product_question"
    },
    {
        "question": "How does compound interest work?",
        "context": "Investment fundamentals and savings growth",
        "ground_truth": "Compound interest is when you earn interest on both your initial principal and the accumulated interest from previous periods, allowing your money to grow exponentially over time.",
        "use_case": "product_question"
    },
    {
        "question": "What is the difference between a traditional IRA and a Roth IRA?",
        "context": "Individual retirement accounts and tax treatment",
        "ground_truth": "A traditional IRA offers tax-deductible contributions with taxed withdrawals in retirement, while a Roth IRA uses after-tax contributions but provides tax-free withdrawals in retirement.",
        "use_case": "product_question"
    },
    {
        "question": "What are the FDIC insurance limits?",
        "context": "Banking regulations and deposit insurance",
        "ground_truth": "The FDIC insures deposits up to $250,000 per depositor, per insured bank, for each account ownership category, protecting customers if a bank fails.",
        "use_case": "compliance"
    },
    {
        "question": "How do I update my beneficiary information?",
        "context": "Account management and customer service",
        "ground_truth": "You can update your beneficiary information by logging into your account online, calling customer service at 1-800-555-1234, or visiting a branch with valid identification.",
        "use_case": "account_inquiry"
    },
    {
        "question": "What is dollar-cost averaging?",
        "context": "Investment strategies",
        "ground_truth": "Dollar-cost averaging is an investment strategy where you invest a fixed amount of money at regular intervals regardless of market conditions, helping to reduce the impact of volatility.",
        "use_case": "product_question"
    },
    {
        "question": "What disclosures are required for securities trading?",
        "context": "Regulatory compliance and securities law",
        "ground_truth": "Securities trading requires disclosure of material information, risks, fees, conflicts of interest, and compliance with SEC regulations to ensure investor protection and market transparency.",
        "use_case": "compliance"
    }
]

def calculate_similarity(output: str, ground_truth: str) -> float:
    """
    Calculate semantic similarity between model output and ground truth.
    
    This is a simplified implementation using word overlap. In production,
    consider using embeddings-based similarity or BERT-based metrics.
    
    Args:
        output: The model's generated text
        ground_truth: The expected answer
        
    Returns:
        Similarity score between 0 and 1
    """
    if not output or not ground_truth:
        return 0.0
    
    # Normalize text
    output_words = set(output.lower().split())
    truth_words = set(ground_truth.lower().split())
    
    if not truth_words:
        return 0.0
    
    # Calculate Jaccard similarity
    common_words = output_words.intersection(truth_words)
    union_words = output_words.union(truth_words)
    
    jaccard_score = len(common_words) / len(union_words) if union_words else 0.0
    
    # Calculate recall (how much of ground truth is covered)
    recall_score = len(common_words) / len(truth_words)
    
    # Weighted combination (favor recall for comprehensiveness)
    similarity = 0.4 * jaccard_score + 0.6 * recall_score
    
    return round(similarity, 3)

print(f"✓ Created {len(FINANCIAL_TEST_CASES)} financial domain test cases")
print("✓ Similarity calculation function implemented")
print("\nTest case categories:")
for use_case in set(tc['use_case'] for tc in FINANCIAL_TEST_CASES):
    count = sum(1 for tc in FINANCIAL_TEST_CASES if tc['use_case'] == use_case)
    print(f"  - {use_case}: {count} cases")

✓ Created 7 financial domain test cases
✓ Similarity calculation function implemented

Test case categories:
  - product_question: 4 cases
  - account_inquiry: 1 cases
  - compliance: 2 cases


### Section 4: Run Model Evaluation

Execute comprehensive benchmarking across all models and test cases.

In [5]:
def evaluate_models() -> pd.DataFrame:
    """
    Evaluate all models on all test cases and return results DataFrame.
    
    Returns:
        DataFrame with evaluation results including latency, quality, and errors
    """
    results = []
    total_evaluations = len(MODELS_TO_EVALUATE) * len(FINANCIAL_TEST_CASES)
    current = 0
    
    print(f"Starting evaluation of {len(MODELS_TO_EVALUATE)} models on {len(FINANCIAL_TEST_CASES)} test cases...")
    print(f"Total evaluations: {total_evaluations}\n")
    
    for test_case in FINANCIAL_TEST_CASES:
        # Construct prompt with context
        prompt = f"""Context: {test_case['context']}

Question: {test_case['question']}

Please provide a clear, accurate answer suitable for a financial services customer."""
        
        for model_id in MODELS_TO_EVALUATE:
            current += 1
            print(f"[{current}/{total_evaluations}] Evaluating {model_id.split('.')[-1]} on: {test_case['question'][:50]}...")
            
            # Invoke the model
            response = invoke_model(model_id, prompt)
            
            if response["success"]:
                # Calculate similarity with ground truth
                similarity = calculate_similarity(
                    response["output"], 
                    test_case["ground_truth"]
                )
                
                # Estimate cost (simplified - actual costs vary by model)
                cost_per_1k_tokens = 0.002 if "claude-3" in model_id else 0.001
                estimated_cost = (response["token_count"] / 1000) * cost_per_1k_tokens
                
                results.append({
                    "model_id": model_id,
                    "model_name": model_id.split('.')[-1],
                    "question": test_case["question"],
                    "use_case": test_case["use_case"],
                    "output": response["output"],
                    "ground_truth": test_case["ground_truth"],
                    "latency_ms": round(response["latency"] * 1000, 2),
                    "token_count": response["token_count"],
                    "similarity_score": similarity,
                    "estimated_cost": round(estimated_cost, 6),
                    "success": True,
                    "error": None,
                    "timestamp": datetime.now().isoformat()
                })
            else:
                results.append({
                    "model_id": model_id,
                    "model_name": model_id.split('.')[-1],
                    "question": test_case["question"],
                    "use_case": test_case["use_case"],
                    "output": None,
                    "ground_truth": test_case["ground_truth"],
                    "latency_ms": round(response["latency"] * 1000, 2),
                    "token_count": 0,
                    "similarity_score": 0.0,
                    "estimated_cost": 0.0,
                    "success": False,
                    "error": response["error"],
                    "timestamp": datetime.now().isoformat()
                })
            
            # Small delay to avoid throttling
            time.sleep(0.5)
    
    print("\n✓ Evaluation complete!")
    return pd.DataFrame(results)

# Run the evaluation
results_df = evaluate_models()

# Display summary statistics
print("\n" + "="*80)
print("EVALUATION SUMMARY")
print("="*80)

summary = results_df.groupby("model_name").agg({
    "latency_ms": ["mean", "std", "min", "max"],
    "similarity_score": ["mean", "std"],
    "token_count": "mean",
    "estimated_cost": "sum",
    "success": "sum"
}).round(3)

print(summary)
print("\nSuccess rates:")
print(results_df.groupby("model_name")["success"].value_counts())

Starting evaluation of 3 models on 7 test cases...
Total evaluations: 21

[1/21] Evaluating claude-3-sonnet-20240229-v1:0 on: What is a 401(k) retirement plan?...
[2/21] Evaluating claude-instant-v1 on: What is a 401(k) retirement plan?...
[3/21] Evaluating titan-text-express-v1 on: What is a 401(k) retirement plan?...
[4/21] Evaluating claude-3-sonnet-20240229-v1:0 on: How does compound interest work?...
[5/21] Evaluating claude-instant-v1 on: How does compound interest work?...
[6/21] Evaluating titan-text-express-v1 on: How does compound interest work?...
[7/21] Evaluating claude-3-sonnet-20240229-v1:0 on: What is the difference between a traditional IRA a...
[8/21] Evaluating claude-instant-v1 on: What is the difference between a traditional IRA a...
[9/21] Evaluating titan-text-express-v1 on: What is the difference between a traditional IRA a...
[10/21] Evaluating claude-3-sonnet-20240229-v1:0 on: What are the FDIC insurance limits?...
[11/21] Evaluating claude-instant-v1 on: What

### Section 5: Analyze Results and Create Selection Strategy

Process evaluation results to create an intelligent model selection strategy.

In [6]:
def create_model_selection_strategy(results_df: pd.DataFrame) -> Dict[str, Any]:
    """
    Create a model selection strategy based on evaluation results.
    
    Analyzes quality, latency, and cost to rank models and assign them
    to specific use cases.
    
    Args:
        results_df: DataFrame with evaluation results
        
    Returns:
        Strategy dictionary for AppConfig
    """
    # Filter successful evaluations only
    successful_df = results_df[results_df["success"] == True].copy()
    
    # Calculate aggregate metrics by model
    model_scores = successful_df.groupby("model_id").agg({
        "latency_ms": "mean",
        "similarity_score": "mean",
        "estimated_cost": "sum",
        "success": "count"
    }).reset_index()
    
    model_scores.columns = ["model_id", "avg_latency_ms", "avg_similarity", "total_cost", "num_evaluations"]
    
    # Normalize scores (0-1 scale)
    if len(model_scores) > 0:
        max_latency = model_scores["avg_latency_ms"].max()
        max_cost = model_scores["total_cost"].max()
        
        # Lower latency is better, so invert
        model_scores["latency_score"] = 1 - (model_scores["avg_latency_ms"] / max_latency)
        
        # Lower cost is better, so invert
        model_scores["cost_score"] = 1 - (model_scores["total_cost"] / max_cost) if max_cost > 0 else 1.0
        
        # Higher similarity is better (already 0-1)
        model_scores["quality_score"] = model_scores["avg_similarity"]
        
        # Calculate weighted overall score
        # Weights: Quality 50%, Latency 30%, Cost 20% (adjust for your priorities)
        model_scores["overall_score"] = (
            0.50 * model_scores["quality_score"] +
            0.30 * model_scores["latency_score"] +
            0.20 * model_scores["cost_score"]
        )
        
        # Sort by overall score
        model_scores = model_scores.sort_values("overall_score", ascending=False)
    
    # Determine use case specific models
    use_case_models = {}
    
    for use_case in successful_df["use_case"].unique():
        use_case_df = successful_df[successful_df["use_case"] == use_case]
        use_case_scores = use_case_df.groupby("model_id")["similarity_score"].mean()
        best_model = use_case_scores.idxmax()
        use_case_models[use_case] = best_model
    
    # Create strategy
    strategy = {
        "version": "1.0",
        "last_updated": datetime.now().isoformat(),
        "primary_model": model_scores.iloc[0]["model_id"] if len(model_scores) > 0 else MODELS_TO_EVALUATE[0],
        "fallback_models": model_scores.iloc[1:]["model_id"].tolist() if len(model_scores) > 1 else [],
        "use_case_models": use_case_models,
        "model_scores": model_scores.to_dict(orient="records"),
        "thresholds": {
            "max_latency_ms": 5000,
            "min_similarity_score": 0.3,
            "max_cost_per_request": 0.01
        },
        "guardrails": {
            "content_filters": ["profanity", "pii", "financial_advice_disclaimer"],
            "max_retries": 3,
            "circuit_breaker_threshold": 5
        }
    }
    
    return strategy

# Create the strategy
strategy = create_model_selection_strategy(results_df)

# Display the strategy
print("\n" + "="*80)
print("MODEL SELECTION STRATEGY")
print("="*80)
print(f"\nPrimary Model: {strategy['primary_model']}")
print(f"Fallback Models: {', '.join(strategy['fallback_models'])}")
print("\nUse Case Assignments:")
for use_case, model in strategy['use_case_models'].items():
    print(f"  - {use_case}: {model.split('.')[-1]}")

# Save results and strategy to files
results_df.to_csv("model_evaluation_results.csv", index=False)
print("\n✓ Saved evaluation results to: model_evaluation_results.csv")

with open("model_selection_strategy.json", "w") as f:
    json.dump(strategy, f, indent=2)
print("✓ Saved selection strategy to: model_selection_strategy.json")


MODEL SELECTION STRATEGY

Primary Model: amazon.titan-text-express-v1
Fallback Models: anthropic.claude-3-sonnet-20240229-v1:0

Use Case Assignments:
  - product_question: claude-3-sonnet-20240229-v1:0
  - compliance: claude-3-sonnet-20240229-v1:0
  - account_inquiry: claude-3-sonnet-20240229-v1:0

✓ Saved evaluation results to: model_evaluation_results.csv
✓ Saved selection strategy to: model_selection_strategy.json


## Part 2: Flexible Architecture for Dynamic Model Selection

### Section 6: Configure AWS AppConfig

Set up AppConfig to manage model selection strategy dynamically.

In [7]:
def setup_appconfig():
    """
    Create AWS AppConfig application, environment, and configuration profile.
    Deploy the model selection strategy.
    """
    try:
        # Create application
        print("Creating AppConfig application...")
        try:
            app_response = appconfig_client.create_application(
                Name=APP_NAME,
                Description="AI Assistant model selection configuration"
            )
            app_id = app_response['Id']
            print(f"✓ Created application: {app_id}")
        except appconfig_client.exceptions.ResourceNotFoundException:
            # Application might already exist
            apps = appconfig_client.list_applications()
            app_id = next((app['Id'] for app in apps['Items'] if app['Name'] == APP_NAME), None)
            if app_id:
                print(f"✓ Using existing application: {app_id}")
            else:
                raise
        
        # Create environment
        print("Creating environment...")
        try:
            env_response = appconfig_client.create_environment(
                ApplicationId=app_id,
                Name=ENVIRONMENT,
                Description=f"{ENVIRONMENT} environment for AI Assistant"
            )
            env_id = env_response['Id']
            print(f"✓ Created environment: {env_id}")
        except:
            # Environment might already exist
            envs = appconfig_client.list_environments(ApplicationId=app_id)
            env_id = next((env['Id'] for env in envs['Items'] if env['Name'] == ENVIRONMENT), None)
            if env_id:
                print(f"✓ Using existing environment: {env_id}")
            else:
                raise
        
        # Create configuration profile
        print("Creating configuration profile...")
        try:
            profile_response = appconfig_client.create_configuration_profile(
                ApplicationId=app_id,
                Name="ModelSelectionStrategy",
                Description="Dynamic model selection and routing strategy",
                LocationUri="hosted",
                Type="AWS.Freeform"
            )
            profile_id = profile_response['Id']
            print(f"✓ Created configuration profile: {profile_id}")
        except:
            # Profile might already exist
            profiles = appconfig_client.list_configuration_profiles(ApplicationId=app_id)
            profile_id = next((p['Id'] for p in profiles['Items'] if p['Name'] == "ModelSelectionStrategy"), None)
            if profile_id:
                print(f"✓ Using existing configuration profile: {profile_id}")
            else:
                raise
        
        # Create hosted configuration version
        print("Creating configuration version...")
        with open("model_selection_strategy.json", "r") as f:
            config_content = f.read()
        
        version_response = appconfig_client.create_hosted_configuration_version(
            ApplicationId=app_id,
            ConfigurationProfileId=profile_id,
            Content=config_content.encode('utf-8'),
            ContentType="application/json",
            Description=f"Model selection strategy - {datetime.now().isoformat()}"
        )
        version_number = version_response['VersionNumber']
        print(f"✓ Created configuration version: {version_number}")
        
        # Create deployment strategy (if needed)
        print("Setting up deployment strategy...")
        try:
            strategy_response = appconfig_client.create_deployment_strategy(
                Name="FastDeployment",
                Description="Deploy configuration changes immediately",
                DeploymentDurationInMinutes=0,
                GrowthFactor=100,
                ReplicateTo="NONE"
            )
            strategy_id = strategy_response['Id']
            print(f"✓ Created deployment strategy: {strategy_id}")
        except:
            # Use predefined strategy
            strategy_id = "AppConfig.AllAtOnce"
            print(f"✓ Using predefined deployment strategy: {strategy_id}")
        
        print("\n✓ AppConfig setup complete!")
        
        return {
            "application_id": app_id,
            "environment_id": env_id,
            "configuration_profile_id": profile_id,
            "deployment_strategy_id": strategy_id
        }
        
    except Exception as e:
        print(f"✗ Error setting up AppConfig: {str(e)}")
        return None

# Set up AppConfig
appconfig_info = setup_appconfig()

if appconfig_info:
    print("\nAppConfig Resource IDs:")
    for key, value in appconfig_info.items():
        print(f"  {key}: {value}")

Creating AppConfig application...
✓ Created application: vb86etf
Creating environment...
✓ Created environment: 28u8itm
Creating configuration profile...
✓ Created configuration profile: 6o43wqr
Creating configuration version...
✓ Created configuration version: 1
Setting up deployment strategy...
✓ Created deployment strategy: g1y56tt

✓ AppConfig setup complete!

AppConfig Resource IDs:
  application_id: vb86etf
  environment_id: 28u8itm
  configuration_profile_id: 6o43wqr
  deployment_strategy_id: g1y56tt


### Section 7: Implement Model Abstraction Lambda

Create the Lambda function that dynamically selects and invokes models.

In [8]:
# Lambda function code for model abstraction
MODEL_ABSTRACTION_LAMBDA_CODE = '''
import boto3
import json
import os
from datetime import datetime

bedrock_runtime = boto3.client('bedrock-runtime')
appconfig_client = boto3.client('appconfig')

# Cache configuration
config_cache = {
    "data": None,
    "timestamp": None,
    "ttl": 300  # 5 minutes
}

def get_configuration():
    """Retrieve configuration from AppConfig with caching."""
    current_time = datetime.now().timestamp()
    
    # Check cache
    if (config_cache["data"] is not None and 
        config_cache["timestamp"] is not None and
        current_time - config_cache["timestamp"] < config_cache["ttl"]):
        return config_cache["data"]
    
    # Fetch fresh configuration
    try:
        response = appconfig_client.get_configuration(
            Application=os.environ['APP_NAME'],
            Environment=os.environ['ENVIRONMENT'],
            Configuration='ModelSelectionStrategy',
            ClientId='model-abstraction-lambda'
        )
        
        config = json.loads(response['Content'].read().decode('utf-8'))
        
        # Update cache
        config_cache["data"] = config
        config_cache["timestamp"] = current_time
        
        return config
    except Exception as e:
        print(f"Error fetching configuration: {str(e)}")
        # Return cached data if available
        return config_cache["data"]

def select_model(config, use_case):
    """Select appropriate model based on configuration and use case."""
    if not config:
        return None
    
    # Check for use case specific model
    use_case_models = config.get('use_case_models', {})
    if use_case in use_case_models:
        return use_case_models[use_case]
    
    # Default to primary model
    return config.get('primary_model')

def invoke_model(model_id, prompt, max_tokens=500):
    """Invoke the selected Bedrock model."""
    try:
        # Prepare request based on model provider
        if "anthropic" in model_id:
            body = json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": max_tokens,
                "messages": [{"role": "user", "content": prompt}],
                "temperature": 0.7,
                "top_p": 0.9
            })
        elif "amazon" in model_id:
            body = json.dumps({
                "inputText": prompt,
                "textGenerationConfig": {
                    "maxTokenCount": max_tokens,
                    "temperature": 0.7,
                    "topP": 0.9
                }
            })
        else:
            raise ValueError(f"Unsupported model: {model_id}")
        
        # Invoke model
        response = bedrock_runtime.invoke_model(
            modelId=model_id,
            body=body
        )
        
        # Parse response
        response_body = json.loads(response['body'].read().decode())
        
        if "anthropic" in model_id:
            output = response_body['content'][0]['text']
        elif "amazon" in model_id:
            output = response_body['results'][0]['outputText']
        
        return {
            "success": True,
            "output": output,
            "model_used": model_id
        }
        
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "model_used": model_id
        }

def lambda_handler(event, context):
    """Main Lambda handler for model abstraction."""
    try:
        # Parse request
        body = json.loads(event.get('body', '{}'))
        prompt = body.get('prompt', '')
        use_case = body.get('use_case', 'general')
        max_tokens = body.get('max_tokens', 500)
        
        # Validate input
        if not prompt:
            return {
                'statusCode': 400,
                'body': json.dumps({'error': 'Prompt is required'})
            }
        
        # Get configuration
        config = get_configuration()
        if not config:
            return {
                'statusCode': 500,
                'body': json.dumps({'error': 'Failed to load configuration'})
            }
        
        # Select model
        model_id = select_model(config, use_case)
        if not model_id:
            return {
                'statusCode': 500,
                'body': json.dumps({'error': 'No model available for use case'})
            }
        
        # Invoke model with retries
        max_retries = config.get('guardrails', {}).get('max_retries', 3)
        
        for attempt in range(max_retries):
            result = invoke_model(model_id, prompt, max_tokens)
            
            if result["success"]:
                return {
                    'statusCode': 200,
                    'body': json.dumps({
                        'response': result["output"],
                        'model_used': result["model_used"],
                        'use_case': use_case,
                        'attempt': attempt + 1
                    }),
                    'headers': {
                        'Content-Type': 'application/json'
                    }
                }
        
        # All retries failed
        return {
            'statusCode': 500,
            'body': json.dumps({
                'error': 'Model invocation failed after retries',
                'model_used': model_id
            })
        }
        
    except Exception as e:
        return {
            'statusCode': 500,
            'body': json.dumps({'error': str(e)})
        }
'''

# Save Lambda code to file
with open("model_abstraction_lambda.py", "w") as f:
    f.write(MODEL_ABSTRACTION_LAMBDA_CODE)

print("✓ Lambda function code saved to: model_abstraction_lambda.py")
print("\nTo deploy this Lambda function:")
print("1. Create a deployment package with dependencies (boto3 is included in Lambda by default)")
print("2. Create IAM role with permissions for Bedrock and AppConfig")
print("3. Deploy using AWS CLI, Console, or SAM/CDK")

✓ Lambda function code saved to: model_abstraction_lambda.py

To deploy this Lambda function:
1. Create a deployment package with dependencies (boto3 is included in Lambda by default)
2. Create IAM role with permissions for Bedrock and AppConfig
3. Deploy using AWS CLI, Console, or SAM/CDK


### Section 8: Deploy API Gateway Integration

Set up API Gateway to expose the Lambda function.

In [10]:
def create_api_gateway_integration(lambda_arn):
    """
    Create API Gateway REST API with Lambda integration.
    
    Args:
        lambda_arn: ARN of the model abstraction Lambda function
        
    Returns:
        Dictionary with API Gateway details
    """
    try:
        # Create REST API
        print("Creating REST API...")
        api_response = apigateway_client.create_rest_api(
            name=f"{APP_NAME}-API",
            description="API for AI Assistant model abstraction",
            endpointConfiguration={'types': ['REGIONAL']}
        )
        api_id = api_response['id']
        print(f"✓ Created API: {api_id}")
        
        # Get root resource
        resources = apigateway_client.get_resources(restApiId=api_id)
        root_id = resources['items'][0]['id']
        
        # Create /generate resource
        print("Creating /generate resource...")
        resource_response = apigateway_client.create_resource(
            restApiId=api_id,
            parentId=root_id,
            pathPart='generate'
        )
        resource_id = resource_response['id']
        print(f"✓ Created resource: {resource_id}")
        
        # Create POST method
        print("Creating POST method...")
        apigateway_client.put_method(
            restApiId=api_id,
            resourceId=resource_id,
            httpMethod='POST',
            authorizationType='NONE',
            apiKeyRequired=False
        )
        print("✓ Created POST method")
        
        # Set up Lambda integration
        print("Setting up Lambda integration...")
        region = boto3.session.Session().region_name
        uri = f"arn:aws:apigateway:{region}:lambda:path/2015-03-31/functions/{lambda_arn}/invocations"
        
        apigateway_client.put_integration(
            restApiId=api_id,
            resourceId=resource_id,
            httpMethod='POST',
            type='AWS_PROXY',
            integrationHttpMethod='POST',
            uri=uri
        )
        print("✓ Configured Lambda integration")
        
        # Create method response
        apigateway_client.put_method_response(
            restApiId=api_id,
            resourceId=resource_id,
            httpMethod='POST',
            statusCode='200',
            responseModels={'application/json': 'Empty'}
        )
        
        # Deploy API
        print(f"Deploying API to {ENVIRONMENT} stage...")
        deployment = apigateway_client.create_deployment(
            restApiId=api_id,
            stageName=ENVIRONMENT,
            description=f"Deployment at {datetime.now().isoformat()}"
        )
        print(f"✓ Deployed API: {deployment['id']}")
        
        # Construct endpoint URL
        endpoint_url = f"https://{api_id}.execute-api.{region}.amazonaws.com/{ENVIRONMENT}/generate"
        
        print("\n✓ API Gateway setup complete!")
        print(f"\nEndpoint URL: {endpoint_url}")
        
        return {
            "api_id": api_id,
            "resource_id": resource_id,
            "deployment_id": deployment['id'],
            "endpoint_url": endpoint_url
        }
        
    except Exception as e:
        print(f"✗ Error creating API Gateway: {str(e)}")
        return None

# Note: This would require an actual Lambda ARN
# For demonstration, we'll show the structure
print("API Gateway Integration Setup")
print("="*80)
print("\nTo complete the setup:")
print("1. Deploy the model_abstraction_lambda.py function to AWS Lambda")
print("2. Note the Lambda ARN")
print("3. Run: create_api_gateway_integration(lambda_arn)")
print("\nExample usage:")
print("  api_info = create_api_gateway_integration('arn:aws:lambda:us-east-1:123456789012:function:ModelAbstraction')")

API Gateway Integration Setup

To complete the setup:
1. Deploy the model_abstraction_lambda.py function to AWS Lambda
2. Note the Lambda ARN
3. Run: create_api_gateway_integration(lambda_arn)

Example usage:
  api_info = create_api_gateway_integration('arn:aws:lambda:us-east-1:123456789012:function:ModelAbstraction')


In [11]:
# Deploy API Gateway integration using the deployed Lambda ARN
lambda_arn = 'arn:aws:lambda:us-east-1:091366569168:function:ModelAbstractionLambda'
api_info = create_api_gateway_integration(lambda_arn)

if api_info:
    print(f"\nAPI Gateway ID: {api_info['api_id']}")
    print(f"Test Command: curl -X POST {api_info['endpoint_url']} -d '{{\"prompt\": \"What is a 401k?\", \"use_case\": \"product_question\"}}'")

Creating REST API...
✓ Created API: qqskzpxhgb
Creating /generate resource...
✓ Created resource: wxx8f9
Creating POST method...
✓ Created POST method
Setting up Lambda integration...
✓ Configured Lambda integration
Deploying API to prod stage...
✓ Deployed API: he6plk

✓ API Gateway setup complete!

Endpoint URL: https://qqskzpxhgb.execute-api.us-east-1.amazonaws.com/prod/generate

API Gateway ID: qqskzpxhgb
Test Command: curl -X POST https://qqskzpxhgb.execute-api.us-east-1.amazonaws.com/prod/generate -d '{"prompt": "What is a 401k?", "use_case": "product_question"}'


- Here is the right CLI:
```powershell
Invoke-RestMethod -Uri "https://qqskzpxhgb.execute-api.us-east-1.amazonaws.com/prod/generate" -Method Post -Body '{"prompt": "What is a 401k?", "use_case": "product_question"}' -ContentType "application/json"
```
- Response:

A 401(k) is a tax-advantaged retirement savings plan offered by many employers in the United States. It allows employees to contribute a portion of their pre-tax salary to individual accounts, which can then be invested in various financial products such as stocks, bonds, and mutual funds. Employers may also offer matching contributions to encourage participation. The funds in a 401(k) grow tax-deferred until withdrawal, typically at retirement age, when they are taxed as ordinary income. Early withdrawals before age 59½ may incur penalties and taxes.



## Part 3: Resilient System Design

### Section 9: Create Step Functions Circuit Breaker

Implement circuit breaker pattern with Step Functions for resilient model invocation.

In [13]:
# Step Functions state machine definition
CIRCUIT_BREAKER_STATE_MACHINE = {
    "Comment": "AI Assistant with Circuit Breaker Pattern for Resilient Model Invocation",
    "StartAt": "TryPrimaryModel",
    "States": {
        "TryPrimaryModel": {
            "Type": "Task",
            "Resource": "arn:aws:states:::lambda:invoke",
            "Parameters": {
                "FunctionName": "${PrimaryModelLambdaArn}",
                "Payload": {
                    "prompt.$": "$.prompt",
                    "use_case.$": "$.use_case",
                    "max_tokens.$": "$.max_tokens"
                }
            },
            "ResultPath": "$.primaryResult",
            "Retry": [
                {
                    "ErrorEquals": [
                        "Lambda.ServiceException",
                        "Lambda.TooManyRequestsException"
                    ],
                    "IntervalSeconds": 2,
                    "MaxAttempts": 2,
                    "BackoffRate": 2.0
                }
            ],
            "Catch": [
                {
                    "ErrorEquals": ["States.ALL"],
                    "ResultPath": "$.primaryError",
                    "Next": "IncrementFailureCount"
                }
            ],
            "Next": "ResetFailureCount"
        },
        "IncrementFailureCount": {
            "Type": "Task",
            "Resource": "arn:aws:states:::dynamodb:updateItem",
            "Parameters": {
                "TableName": "${CircuitBreakerTable}",
                "Key": {
                    "circuitId": {"S": "primary-model"}
                },
                "UpdateExpression": "ADD failureCount :inc SET lastFailure = :timestamp",
                "ExpressionAttributeValues": {
                    ":inc": {"N": "1"},
                    ":timestamp": {"S.$": "$$.State.EnteredTime"}
                }
            },
            "ResultPath": "$.circuitState",
            "Next": "CheckCircuitBreaker"
        },
        "CheckCircuitBreaker": {
            "Type": "Choice",
            "Choices": [
                {
                    "Variable": "$.circuitState.Attributes.failureCount.N",
                    "NumericGreaterThan": 5,
                    "Next": "CircuitOpen"
                }
            ],
            "Default": "TryFallbackModel"
        },
        "CircuitOpen": {
            "Type": "Pass",
            "Result": {
                "status": "circuit_open",
                "message": "Circuit breaker open due to repeated failures"
            },
            "ResultPath": "$.circuitStatus",
            "Next": "GracefulDegradation"
        },
        "TryFallbackModel": {
            "Type": "Task",
            "Resource": "arn:aws:states:::lambda:invoke",
            "Parameters": {
                "FunctionName": "${FallbackModelLambdaArn}",
                "Payload": {
                    "prompt.$": "$.prompt",
                    "use_case.$": "$.use_case",
                    "is_fallback": True
                }
            },
            "ResultPath": "$.fallbackResult",
            "Retry": [
                {
                    "ErrorEquals": [
                        "Lambda.ServiceException",
                        "Lambda.TooManyRequestsException"
                    ],
                    "IntervalSeconds": 1,
                    "MaxAttempts": 2,
                    "BackoffRate": 2.0
                }
            ],
            "Catch": [
                {
                    "ErrorEquals": ["States.ALL"],
                    "ResultPath": "$.fallbackError",
                    "Next": "GracefulDegradation"
                }
            ],
            "Next": "SuccessState"
        },
        "GracefulDegradation": {
            "Type": "Task",
            "Resource": "arn:aws:states:::lambda:invoke",
            "Parameters": {
                "FunctionName": "${DegradationLambdaArn}",
                "Payload": {
                    "prompt.$": "$.prompt",
                    "use_case.$": "$.use_case",
                    "reason": "all_models_failed"
                }
            },
            "ResultPath": "$.degradationResult",
            "Next": "SuccessState"
        },
        "ResetFailureCount": {
            "Type": "Task",
            "Resource": "arn:aws:states:::dynamodb:updateItem",
            "Parameters": {
                "TableName": "${CircuitBreakerTable}",
                "Key": {
                    "circuitId": {"S": "primary-model"}
                },
                "UpdateExpression": "SET failureCount = :zero, lastSuccess = :timestamp",
                "ExpressionAttributeValues": {
                    ":zero": {"N": "0"},
                    ":timestamp": {"S.$": "$$.State.EnteredTime"}
                }
            },
            "ResultPath": "$.resetResult",
            "Next": "SuccessState"
        },
        "SuccessState": {
            "Type": "Succeed"
        }
    }
}

# Save state machine definition
with open("circuit_breaker_state_machine.json", "w") as f:
    json.dump(CIRCUIT_BREAKER_STATE_MACHINE, f, indent=2)

print("✓ Step Functions state machine definition saved to: circuit_breaker_state_machine.json")
print("\nCircuit Breaker Features:")
print("  ✓ Primary model invocation with retries")
print("  ✓ Automatic fallback to secondary model")
print("  ✓ Failure count tracking in DynamoDB")
print("  ✓ Circuit breaker opens after 5 failures")
print("  ✓ Graceful degradation when all models fail")
print("  ✓ Success resets failure count")

✓ Step Functions state machine definition saved to: circuit_breaker_state_machine.json

Circuit Breaker Features:
  ✓ Primary model invocation with retries
  ✓ Automatic fallback to secondary model
  ✓ Failure count tracking in DynamoDB
  ✓ Circuit breaker opens after 5 failures
  ✓ Graceful degradation when all models fail
  ✓ Success resets failure count


### Section 10: Implement Fallback Lambda

Create the fallback Lambda that uses a simpler, more reliable model configuration.

In [14]:
# Fallback model Lambda code
FALLBACK_MODEL_LAMBDA_CODE = '''
import boto3
import json
import os

bedrock_runtime = boto3.client('bedrock-runtime')

def lambda_handler(event, context):
    """
    Fallback model handler that uses a simpler, more reliable model.
    
    This Lambda is invoked when the primary model fails, using
    conservative settings for maximum reliability.
    """
    try:
        # Extract parameters
        prompt = event.get('prompt', '')
        use_case = event.get('use_case', 'general')
        is_fallback = event.get('is_fallback', False)
        
        # Use Titan Express as fallback (simpler, more reliable)
        model_id = "amazon.titan-text-express-v1"
        
        # Conservative parameters for reliability
        body = json.dumps({
            "inputText": prompt,
            "textGenerationConfig": {
                "maxTokenCount": 300,  # Reduced for faster response
                "temperature": 0.5,    # Lower temperature for consistency
                "topP": 0.9,
                "stopSequences": []
            }
        })
        
        # Invoke model
        response = bedrock_runtime.invoke_model(
            modelId=model_id,
            body=body
        )
        
        # Parse response
        response_body = json.loads(response['body'].read().decode())
        output = response_body['results'][0]['outputText']
        
        return {
            'statusCode': 200,
            'body': json.dumps({
                'response': output,
                'model_used': f"FALLBACK:{model_id}",
                'use_case': use_case,
                'is_fallback': True,
                'message': 'Fallback model used due to primary model failure'
            })
        }
        
    except Exception as e:
        # Even fallback failed - let Step Functions handle
        raise Exception(f"Fallback model failed: {str(e)}")
'''

# Save fallback Lambda code
with open("fallback_model_lambda.py", "w") as f:
    f.write(FALLBACK_MODEL_LAMBDA_CODE)

print("✓ Fallback Lambda code saved to: fallback_model_lambda.py")
print("\nFallback Lambda Features:")
print("  ✓ Uses Amazon Titan Express (simpler model)")
print("  ✓ Reduced max tokens (300) for faster response")
print("  ✓ Lower temperature (0.5) for consistency")
print("  ✓ Marks responses with FALLBACK prefix")
print("  ✓ Propagates errors to Step Functions for graceful degradation")

✓ Fallback Lambda code saved to: fallback_model_lambda.py

Fallback Lambda Features:
  ✓ Uses Amazon Titan Express (simpler model)
  ✓ Reduced max tokens (300) for faster response
  ✓ Lower temperature (0.5) for consistency
  ✓ Marks responses with FALLBACK prefix
  ✓ Propagates errors to Step Functions for graceful degradation


### Section 11: Implement Graceful Degradation Lambda

Create the final safety net with predefined regulation-safe responses.

In [15]:
# Graceful degradation Lambda code
GRACEFUL_DEGRADATION_LAMBDA_CODE = '''
import json
from datetime import datetime

def lambda_handler(event, context):
    """
    Graceful degradation handler that returns predefined, regulation-safe responses.
    
    This is the final safety net when all models fail, ensuring customers
    always receive helpful guidance that complies with financial regulations.
    """
    prompt = event.get('prompt', '')
    use_case = event.get('use_case', 'general')
    reason = event.get('reason', 'unknown')
    
    # Regulation-compliant responses by use case
    responses = {
        "general": {
            "response": "I apologize, but I'm currently experiencing technical difficulties and cannot process your request at this time. For immediate assistance, please contact our customer service team at 1-800-555-1234 (available 24/7) or visit your nearest branch location.",
            "contact_info": {
                "phone": "1-800-555-1234",
                "hours": "24/7",
                "alternative": "Visit nearest branch"
            }
        },
        "product_question": {
            "response": "I'm unable to access our product information systems at the moment. For detailed information about our financial products and services, please:\\n\\n1. Call our product specialists at 1-800-555-PROD (1-800-555-7763)\\n2. Visit www.example.com/products\\n3. Speak with a representative at any branch location\\n\\nOur team can provide personalized product recommendations based on your financial needs.",
            "contact_info": {
                "phone": "1-800-555-7763",
                "website": "www.example.com/products"
            }
        },
        "account_inquiry": {
            "response": "For security reasons and to protect your personal information, I'm unable to process account inquiries at this time. Please use one of these secure alternatives:\\n\\n1. Log in to your online banking portal\\n2. Call our secure account services line at 1-800-555-ACCT (1-800-555-2228)\\n3. Visit a branch with valid photo identification\\n\\nFor urgent account matters, our phone representatives are available 24/7.",
            "contact_info": {
                "phone": "1-800-555-2228",
                "online": "Online banking portal",
                "hours": "24/7"
            }
        },
        "compliance": {
            "response": "I apologize, but I cannot provide regulatory or compliance information at this time. For accurate compliance-related inquiries:\\n\\n1. Contact our compliance department at compliance@example.com\\n2. Call our compliance hotline at 1-800-555-CMPL (1-800-555-2675)\\n3. Refer to official disclosures at www.example.com/disclosures\\n\\nAll financial products are subject to terms and conditions. Please consult official documentation.",
            "contact_info": {
                "email": "compliance@example.com",
                "phone": "1-800-555-2675",
                "website": "www.example.com/disclosures"
            }
        },
        "personalized_outreach": {
            "response": "Thank you for your interest. Unfortunately, I'm unable to provide personalized recommendations at this time. To discuss financial solutions tailored to your needs:\\n\\n1. Schedule an appointment with a financial advisor\\n2. Call 1-800-555-ADVS (1-800-555-2387)\\n3. Request a callback at www.example.com/contact\\n\\nOur advisors can help you achieve your financial goals with personalized guidance.",
            "contact_info": {
                "phone": "1-800-555-2387",
                "website": "www.example.com/contact"
            }
        }
    }
    
    # Get appropriate response
    default_response = responses["general"]
    response_data = responses.get(use_case, default_response)
    
    # Add standard disclaimer for financial services
    disclaimer = "\\n\\n---\\nThis is an automated response. Products and services are subject to terms and conditions. Not all products are available in all areas. Member FDIC. Equal Housing Lender."
    
    # Construct final response
    final_response = response_data["response"] + disclaimer
    
    return {
        'statusCode': 200,
        'body': json.dumps({
            'response': final_response,
            'model_used': 'DEGRADED_SERVICE',
            'use_case': use_case,
            'degradation_reason': reason,
            'contact_info': response_data["contact_info"],
            'timestamp': datetime.now().isoformat(),
            'message': 'Service degraded - using predefined response'
        })
    }
'''

# Save degradation Lambda code
with open("graceful_degradation_lambda.py", "w") as f:
    f.write(GRACEFUL_DEGRADATION_LAMBDA_CODE)

print("✓ Graceful degradation Lambda code saved to: graceful_degradation_lambda.py")
print("\nGraceful Degradation Features:")
print("  ✓ Predefined responses by use case")
print("  ✓ Regulation-compliant messaging")
print("  ✓ Contact information for each scenario")
print("  ✓ Standard FDIC and financial disclaimers")
print("  ✓ Marked as DEGRADED_SERVICE for monitoring")
print("  ✓ Always returns 200 status with helpful guidance")

✓ Graceful degradation Lambda code saved to: graceful_degradation_lambda.py

Graceful Degradation Features:
  ✓ Predefined responses by use case
  ✓ Regulation-compliant messaging
  ✓ Contact information for each scenario
  ✓ Standard FDIC and financial disclaimers
  ✓ Marked as DEGRADED_SERVICE for monitoring
  ✓ Always returns 200 status with helpful guidance


### Section 12: Setup Cross-Region Deployment with CloudFormation

Create CloudFormation template for multi-region deployment.

In [16]:
# CloudFormation template for cross-region deployment
CLOUDFORMATION_TEMPLATE = '''
AWSTemplateFormatVersion: '2010-09-09'
Description: 'AI Assistant Cross-Region Deployment for High Availability'

Parameters:
  Environment:
    Type: String
    Default: prod
    AllowedValues:
      - dev
      - staging
      - prod
    Description: Deployment environment
  
  AppName:
    Type: String
    Default: AIAssistantApp
    Description: Application name for resource naming

Resources:
  # IAM Role for Lambda functions
  LambdaExecutionRole:
    Type: AWS::IAM::Role
    Properties:
      RoleName: !Sub "${AppName}-Lambda-Role-${Environment}"
      AssumeRolePolicyDocument:
        Version: '2012-10-17'
        Statement:
          - Effect: Allow
            Principal:
              Service: lambda.amazonaws.com
            Action: 'sts:AssumeRole'
      ManagedPolicyArns:
        - 'arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
      Policies:
        - PolicyName: BedrockAccess
          PolicyDocument:
            Version: '2012-10-17'
            Statement:
              - Effect: Allow
                Action:
                  - 'bedrock:InvokeModel'
                  - 'bedrock-runtime:InvokeModel'
                Resource: '*'
        - PolicyName: AppConfigAccess
          PolicyDocument:
            Version: '2012-10-17'
            Statement:
              - Effect: Allow
                Action:
                  - 'appconfig:GetConfiguration'
                  - 'appconfig:StartConfigurationSession'
                  - 'appconfig:GetLatestConfiguration'
                Resource: '*'
        - PolicyName: DynamoDBAccess
          PolicyDocument:
            Version: '2012-10-17'
            Statement:
              - Effect: Allow
                Action:
                  - 'dynamodb:PutItem'
                  - 'dynamodb:GetItem'
                  - 'dynamodb:UpdateItem'
                Resource: !GetAtt CircuitBreakerTable.Arn

  # DynamoDB table for circuit breaker state
  CircuitBreakerTable:
    Type: AWS::DynamoDB::Table
    Properties:
      TableName: !Sub "${AppName}-CircuitBreaker-${Environment}"
      BillingMode: PAY_PER_REQUEST
      AttributeDefinitions:
        - AttributeName: circuitId
          AttributeType: S
      KeySchema:
        - AttributeName: circuitId
          KeyType: HASH
      StreamSpecification:
        StreamViewType: NEW_AND_OLD_IMAGES
      PointInTimeRecoverySpecification:
        PointInTimeRecoveryEnabled: true

  # Model Abstraction Lambda
  ModelAbstractionLambda:
    Type: AWS::Lambda::Function
    Properties:
      FunctionName: !Sub "${AppName}-ModelAbstraction-${Environment}"
      Runtime: python3.11
      Handler: index.lambda_handler
      Role: !GetAtt LambdaExecutionRole.Arn
      Timeout: 30
      MemorySize: 512
      Environment:
        Variables:
          APP_NAME: !Ref AppName
          ENVIRONMENT: !Ref Environment
      Code:
        ZipFile: |
          # Code from model_abstraction_lambda.py
          import json
          def lambda_handler(event, context):
              return {'statusCode': 200, 'body': json.dumps('Placeholder')}

  # Fallback Model Lambda
  FallbackModelLambda:
    Type: AWS::Lambda::Function
    Properties:
      FunctionName: !Sub "${AppName}-FallbackModel-${Environment}"
      Runtime: python3.11
      Handler: index.lambda_handler
      Role: !GetAtt LambdaExecutionRole.Arn
      Timeout: 20
      MemorySize: 256
      Code:
        ZipFile: |
          # Code from fallback_model_lambda.py
          import json
          def lambda_handler(event, context):
              return {'statusCode': 200, 'body': json.dumps('Fallback')}

  # Graceful Degradation Lambda
  GracefulDegradationLambda:
    Type: AWS::Lambda::Function
    Properties:
      FunctionName: !Sub "${AppName}-GracefulDegradation-${Environment}"
      Runtime: python3.11
      Handler: index.lambda_handler
      Role: !GetAtt LambdaExecutionRole.Arn
      Timeout: 10
      MemorySize: 128
      Code:
        ZipFile: |
          # Code from graceful_degradation_lambda.py
          import json
          def lambda_handler(event, context):
              return {'statusCode': 200, 'body': json.dumps('Degraded')}

  # API Gateway
  ApiGateway:
    Type: AWS::ApiGateway::RestApi
    Properties:
      Name: !Sub "${AppName}-API-${Environment}"
      Description: API for AI Assistant
      EndpointConfiguration:
        Types:
          - REGIONAL

  # API Gateway Resource
  GenerateResource:
    Type: AWS::ApiGateway::Resource
    Properties:
      RestApiId: !Ref ApiGateway
      ParentId: !GetAtt ApiGateway.RootResourceId
      PathPart: generate

  # API Gateway Method
  GenerateMethod:
    Type: AWS::ApiGateway::Method
    Properties:
      RestApiId: !Ref ApiGateway
      ResourceId: !Ref GenerateResource
      HttpMethod: POST
      AuthorizationType: NONE
      Integration:
        Type: AWS_PROXY
        IntegrationHttpMethod: POST
        Uri: !Sub "arn:aws:apigateway:${AWS::Region}:lambda:path/2015-03-31/functions/${ModelAbstractionLambda.Arn}/invocations"

  # Lambda Permission for API Gateway
  ApiGatewayInvokePermission:
    Type: AWS::Lambda::Permission
    Properties:
      FunctionName: !Ref ModelAbstractionLambda
      Action: lambda:InvokeFunction
      Principal: apigateway.amazonaws.com
      SourceArn: !Sub "arn:aws:execute-api:${AWS::Region}:${AWS::AccountId}:${ApiGateway}/*"

  # API Gateway Deployment
  ApiDeployment:
    Type: AWS::ApiGateway::Deployment
    DependsOn: GenerateMethod
    Properties:
      RestApiId: !Ref ApiGateway
      StageName: !Ref Environment

  # CloudWatch Log Group
  ApiLogGroup:
    Type: AWS::Logs::LogGroup
    Properties:
      LogGroupName: !Sub "/aws/apigateway/${AppName}-${Environment}"
      RetentionInDays: 30

Outputs:
  ApiEndpoint:
    Description: API Gateway endpoint URL
    Value: !Sub "https://${ApiGateway}.execute-api.${AWS::Region}.amazonaws.com/${Environment}/generate"
    Export:
      Name: !Sub "${AppName}-ApiEndpoint-${Environment}"
  
  ModelAbstractionLambdaArn:
    Description: Model Abstraction Lambda ARN
    Value: !GetAtt ModelAbstractionLambda.Arn
    Export:
      Name: !Sub "${AppName}-ModelAbstractionLambda-${Environment}"
  
  FallbackModelLambdaArn:
    Description: Fallback Model Lambda ARN
    Value: !GetAtt FallbackModelLambda.Arn
    Export:
      Name: !Sub "${AppName}-FallbackModelLambda-${Environment}"
  
  GracefulDegradationLambdaArn:
    Description: Graceful Degradation Lambda ARN
    Value: !GetAtt GracefulDegradationLambda.Arn
    Export:
      Name: !Sub "${AppName}-GracefulDegradationLambda-${Environment}"
  
  CircuitBreakerTableName:
    Description: DynamoDB Circuit Breaker Table Name
    Value: !Ref CircuitBreakerTable
    Export:
      Name: !Sub "${AppName}-CircuitBreakerTable-${Environment}"
'''

# Save CloudFormation template
with open("cross_region_deployment.yaml", "w") as f:
    f.write(CLOUDFORMATION_TEMPLATE)

print("✓ CloudFormation template saved to: cross_region_deployment.yaml")
print("\nTo deploy to multiple regions:")
print(f"\n# Deploy to {PRIMARY_REGION} (primary):")
print(f"aws cloudformation deploy \\")
print(f"  --template-file cross_region_deployment.yaml \\")
print(f"  --stack-name ai-assistant-stack \\")
print(f"  --parameter-overrides Environment={ENVIRONMENT} \\")
print(f"  --region {PRIMARY_REGION} \\")
print(f"  --capabilities CAPABILITY_NAMED_IAM")
print(f"\n# Deploy to {SECONDARY_REGION} (secondary):")
print(f"aws cloudformation deploy \\")
print(f"  --template-file cross_region_deployment.yaml \\")
print(f"  --stack-name ai-assistant-stack \\")
print(f"  --parameter-overrides Environment={ENVIRONMENT} \\")
print(f"  --region {SECONDARY_REGION} \\")
print(f"  --capabilities CAPABILITY_NAMED_IAM")

✓ CloudFormation template saved to: cross_region_deployment.yaml

To deploy to multiple regions:

# Deploy to us-east-1 (primary):
aws cloudformation deploy \
  --template-file cross_region_deployment.yaml \
  --stack-name ai-assistant-stack \
  --parameter-overrides Environment=prod \
  --region us-east-1 \
  --capabilities CAPABILITY_NAMED_IAM

# Deploy to us-west-2 (secondary):
aws cloudformation deploy \
  --template-file cross_region_deployment.yaml \
  --stack-name ai-assistant-stack \
  --parameter-overrides Environment=prod \
  --region us-west-2 \
  --capabilities CAPABILITY_NAMED_IAM


### Section 13: Configure Route 53 Failover Routing

Set up DNS-based failover for automatic traffic redirection.

In [17]:
def configure_route53_failover(primary_api_id, secondary_api_id, domain_name):
    """
    Configure Route 53 DNS failover for high availability.
    
    Args:
        primary_api_id: API Gateway ID in primary region
        secondary_api_id: API Gateway ID in secondary region
        domain_name: Root domain name (e.g., example.com)
    """
    try:
        # API Gateway Hosted Zone IDs (Regional)
        # https://docs.aws.amazon.com/general/latest/gr/apigateway.html
        API_GATEWAY_ZONES = {
            'us-east-1': 'Z1UJRXOUMOOFQ8',
            'us-west-2': 'Z2OJLYMUO9EFXC',
            'eu-west-1': 'ZLY8HYME6SFDD',
            # Add others as needed
        }
        
        timestamp = str(int(time.time()))
        caller_ref = f"health-check-{timestamp}"
        
        # 1. Create Health Check for Primary Region
        print(f"Creating health check for {PRIMARY_REGION}...")
        fqdn = f"{primary_api_id}.execute-api.{PRIMARY_REGION}.amazonaws.com"
        
        hc_response = route53_client.create_health_check(
            CallerReference=caller_ref,
            HealthCheckConfig={
                'Port': 443,
                'Type': 'HTTPS',
                'ResourcePath': '/prod/generate',
                'FullyQualifiedDomainName': fqdn,
                'RequestInterval': 30,
                'FailureThreshold': 3,
                'MeasureLatency': True,
                'Inverted': False,
                'Disabled': False,
                'EnableSNI': True
            }
        )
        health_check_id = hc_response['HealthCheck']['Id']
        print(f"✓ Created health check: {health_check_id}")
        
        # 2. Get or Create Hosted Zone
        print(f"Checking hosted zone for {domain_name}...")
        hosted_zone_id = None
        
        # List zones to find existing
        zones = route53_client.list_hosted_zones_by_name(DNSName=domain_name)
        # Check if we found an exact match or parent
        for zone in zones['HostedZones']:
            if zone['Name'].rstrip('.') == domain_name.rstrip('.'):
                hosted_zone_id = zone['Id']
                print(f"✓ Found existing hosted zone: {hosted_zone_id}")
                break
        
        if not hosted_zone_id:
            # Create new zone
            print("Creating new hosted zone...")
            hz_response = route53_client.create_hosted_zone(
                Name=domain_name,
                CallerReference=f"zone-{timestamp}",
                HostedZoneConfig={'Comment': 'AI Assistant Failover'}
            )
            hosted_zone_id = hz_response['HostedZone']['Id']
            print(f"✓ Created hosted zone: {hosted_zone_id}")
            
        # 3. Create Failover Records
        print("Creating failover DNS records...")
        
        subdomain = f"ai-assistant.{domain_name}"
        
        change_batch = {
            'Changes': [
                {
                    'Action': 'UPSERT',
                    'ResourceRecordSet': {
                        'Name': subdomain,
                        'Type': 'A',
                        'SetIdentifier': 'Primary',
                        'Failover': 'PRIMARY',
                        'HealthCheckId': health_check_id,
                        'AliasTarget': {
                            'HostedZoneId': API_GATEWAY_ZONES.get(PRIMARY_REGION),
                            'DNSName': f"{primary_api_id}.execute-api.{PRIMARY_REGION}.amazonaws.com",
                            'EvaluateTargetHealth': True
                        }
                    }
                },
                {
                    'Action': 'UPSERT',
                    'ResourceRecordSet': {
                        'Name': subdomain,
                        'Type': 'A',
                        'SetIdentifier': 'Secondary',
                        'Failover': 'SECONDARY',
                        'AliasTarget': {
                            'HostedZoneId': API_GATEWAY_ZONES.get(SECONDARY_REGION),
                            'DNSName': f"{secondary_api_id}.execute-api.{SECONDARY_REGION}.amazonaws.com",
                            'EvaluateTargetHealth': True
                        }
                    }
                }
            ]
        }
        
        route53_client.change_resource_record_sets(
            HostedZoneId=hosted_zone_id,
            ChangeBatch=change_batch
        )
        print("✓ Created failover DNS records")
        
        print(f"\nFailover Setup Complete!")
        print(f"Primary: {PRIMARY_REGION} (Health Check: {health_check_id})")
        print(f"Secondary: {SECONDARY_REGION}")
        print(f"Global Endpoint: https://{subdomain}/prod/generate")
        
        return {
            "health_check_id": health_check_id,
            "hosted_zone_id": hosted_zone_id,
            "domain_name": subdomain
        }
        
    except Exception as e:
        print(f"✗ Error setting up Route 53: {str(e)}")
        return None

# Example usage
print("Route 53 Failover Configuration")
print("="*80)
print("To configure failover:")
print("1. Ensure you have a registered domain name")
print("2. Deploy API Gateway to both regions")
print("3. Run the configuration function:")
print("\nExample:")
print("configure_route53_failover(")
print(f"    primary_api_id='{api_info['api_id'] if 'api_info' in locals() and api_info else 'PRIMARY_API_ID'}',")
print("    secondary_api_id='SECONDARY_API_ID',")
print("    domain_name='yourdomain.com'")
print(")")

Route 53 Failover Configuration
To configure failover:
1. Ensure you have a registered domain name
2. Deploy API Gateway to both regions
3. Run the configuration function:

Example:
configure_route53_failover(
    primary_api_id='qqskzpxhgb',
    secondary_api_id='SECONDARY_API_ID',
    domain_name='yourdomain.com'
)


## Part 4: Model customization and lifecycle management

### Step 1: Fine-tune a model with SageMaker

1. Prepare a fine-tuning dataset:

In [18]:
import pandas as pd
import json

# Create a financial Q&A dataset
data = [
    {"question": "What is a 401(k)?", "answer": "A 401(k) is a tax-advantaged retirement savings plan offered by employers."},
    {"question": "How does compound interest work?", "answer": "Compound interest is when you earn interest on both the money you've saved and the interest you earn."},
    {"question": "What is an ETF?", "answer": "An ETF (Exchange Traded Fund) is a type of security that tracks an index, sector, commodity, or other asset, but which can be purchased or sold on a stock exchange the same way a regular stock can."},
    {"question": "What is a dividend?", "answer": "A dividend is a distribution of some of a company's earnings to a class of its shareholders, as determined by the company's board of directors."},
    {"question": "What is a credit score?", "answer": "A credit score is a numerical expression based on a level analysis of a person's credit files, to represent the creditworthiness of an individual."}
]

# Convert to DataFrame
df = pd.DataFrame(data)

# Save to CSV for SageMaker training
df.to_csv("financial_qa_dataset.csv", index=False)
print("✓ Created financial_qa_dataset.csv")

# Create SageMaker training script
with open("train.py", "w") as f:
    f.write("""
import argparse
import os
import json
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--training-dir", type=str, default=os.environ.get("SM_CHANNEL_TRAINING"))
    return parser.parse_args()

if __name__ == "__main__":
    args = parse_args()
    
    # Load dataset
    data_path = os.path.join(args.training_dir, "financial_qa_dataset.csv")
    df = pd.read_csv(data_path)
    
    # Prepare dataset
    def format_instruction(row):
        return f"Question: {row['question']}\\nAnswer: {row['answer']}"
    
    df["text"] = df.apply(format_instruction, axis=1)
    dataset = Dataset.from_pandas(df[["text"]])
    
    # Load model and tokenizer
    model_name = "distilgpt2"  # Use a smaller model for example purposes
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    model = AutoModelForCausalLM.from_pretrained(model_name)
    
    # Tokenize dataset
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)
    
    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir=args.model_dir,
        num_train_epochs=3,
        per_device_train_batch_size=4,
        save_steps=500,
        save_total_limit=2,
    )
    
    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets,
    )
    
    # Train
    trainer.train()
    
    # Save model
    trainer.save_model(args.model_dir)
    tokenizer.save_pretrained(args.model_dir)
""")

print("✓ Created train.py training script")

✓ Created financial_qa_dataset.csv
✓ Created train.py training script


2. Launch SageMaker Training Job:

Upload the dataset to S3 and define the SageMaker Estimator to run the training script.

In [14]:
import sagemaker
from sagemaker.huggingface import HuggingFace
import boto3

# Initialize SageMaker session
sess = sagemaker.Session()

# Try to get role
try:
    role = sagemaker.get_execution_role()
except ValueError:
    print("Could not get execution role from environment. Searching IAM roles...")
    try:
        iam = boto3.client('iam')
        roles = iam.list_roles()
        # Look for a role with 'SageMaker' and 'Execution' in the name
        sm_roles = [r['Arn'] for r in roles['Roles'] if 'SageMaker' in r['RoleName'] and 'Execution' in r['RoleName']]
        if sm_roles:
            role = sm_roles[0]
            print(f"Found SageMaker execution role: {role}")
        else:
            # Fallback to any SageMaker role
            sm_roles = [r['Arn'] for r in roles['Roles'] if 'SageMaker' in r['RoleName']]
            if sm_roles:
                role = sm_roles[0]
                print(f"Found SageMaker role: {role}")
            else:
                print("No SageMaker role found. Please create one and specify 'role' manually.")
                role = None
    except Exception as e:
        print(f"Error searching roles: {e}")
        role = None

if role:
    print(f"✓ Using role: {role}")
    
    # Parse S3 bucket and prefix
    if "/" in S3_BUCKET:
        bucket_name = S3_BUCKET.split("/")[0]
        prefix = "/".join(S3_BUCKET.split("/")[1:])
        if prefix.endswith("/"):
            prefix = prefix[:-1]
    else:
        bucket_name = S3_BUCKET
        prefix = "financial-qa"
        
    print(f"✓ Uploading data to bucket: {bucket_name}, prefix: {prefix}/data")

    # Upload dataset to S3
    input_data_uri = sess.upload_data(
        path="financial_qa_dataset.csv",
        bucket=bucket_name,
        key_prefix=f"{prefix}/data"
    )
    print(f"✓ Dataset uploaded to: {input_data_uri}")

    print("\n" + "="*80)
    print("SAGEMAKER FINE-TUNING SETUP COMPLETE")
    print("="*80)
    
    print("\n📋 Summary:")
    print(f"   • Training Data: {input_data_uri}")
    print(f"   • Training Script: train.py")
    print(f"   • IAM Role: {role.split('/')[-1]}")
    
    print("\n⚠️  Note: Due to SDK version constraints and instance quota limits,")
    print("   actual training job execution requires either:")
    print("   1. GPU instance quota (ml.g4dn.xlarge)")
    print("   2. Upgraded SageMaker SDK (pip install -U sagemaker)")
    
    print("\n📝 To run the training job manually:")
    print("\n   1. Request GPU quota increase via AWS Service Quotas")
    print("   2. Use GPU-compatible configuration:")
    
    print("""
   huggingface_estimator = HuggingFace(
       entry_point="train.py",
       source_dir=".",
       instance_type="ml.g4dn.xlarge",  # GPU instance
       instance_count=1,
       role=role,
       transformers_version="4.17.0",
       pytorch_version="1.10.2",
       py_version="py38",
       hyperparameters={
           "epochs": 3,
           "train_batch_size": 4,
           "model_dir": "/opt/ml/model"
       }
   )
   
   huggingface_estimator.fit({'training': input_data_uri})
    """)
    
    print("\n✓ All preparation steps completed successfully!")
    print("✓ Training data uploaded and ready for fine-tuning")
    
else:
    print("❌ Skipping SageMaker setup due to missing role.")
    print("   Please create a SageMaker execution role in IAM console.")

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\DELL\AppData\Local\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\DELL\AppData\Local\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\DELL\AppData\Local\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\DELL\AppData\Local\sagemaker\sagemaker\config.yaml


Could not get execution role from environment. Searching IAM roles...
Found SageMaker execution role: arn:aws:iam::091366569168:role/service-role/AmazonSageMaker-ExecutionRole-20230424T121108
✓ Using role: arn:aws:iam::091366569168:role/service-role/AmazonSageMaker-ExecutionRole-20230424T121108
✓ Uploading data to bucket: cert-genai-dev, prefix: bonus_1_2/data
Found SageMaker execution role: arn:aws:iam::091366569168:role/service-role/AmazonSageMaker-ExecutionRole-20230424T121108
✓ Using role: arn:aws:iam::091366569168:role/service-role/AmazonSageMaker-ExecutionRole-20230424T121108
✓ Uploading data to bucket: cert-genai-dev, prefix: bonus_1_2/data
✓ Dataset uploaded to: s3://cert-genai-dev/bonus_1_2/data/financial_qa_dataset.csv

SAGEMAKER FINE-TUNING SETUP COMPLETE

📋 Summary:
   • Training Data: s3://cert-genai-dev/bonus_1_2/data/financial_qa_dataset.csv
   • Training Script: train.py
   • IAM Role: AmazonSageMaker-ExecutionRole-20230424T121108

⚠️  Note: Due to SDK version constrain

### Step 2: Deploy the Fine-tuned Model

Once training is complete, deploy the model to a SageMaker endpoint for inference.

In [ ]:
def deploy_fine_tuned_model(training_job_name=None):
    """
    Deploy a fine-tuned model to a SageMaker endpoint.
    
    Args:
        training_job_name: Name of the completed training job
        
    Returns:
        Dictionary with endpoint information
    """
    try:
        # If training job name provided, get the model artifacts
        if training_job_name:
            print(f"Retrieving model artifacts from training job: {training_job_name}")
            
            # Describe training job to get model artifacts
            training_job = sagemaker_client.describe_training_job(
                TrainingJobName=training_job_name
            )
            
            model_data_url = training_job['ModelArtifacts']['S3ModelArtifacts']
            print(f"✓ Model artifacts: {model_data_url}")
            
            # Create model
            model_name = f"financial-qa-model-{int(time.time())}"
            print(f"Creating model: {model_name}")
            
            # Get execution role
            try:
                role = sagemaker.get_execution_role()
            except:
                iam = boto3.client('iam')
                roles = iam.list_roles()
                sm_roles = [r['Arn'] for r in roles['Roles'] if 'SageMaker' in r['RoleName']]
                role = sm_roles[0] if sm_roles else None
            
            if not role:
                print("❌ No SageMaker role found")
                return None
            
            # Create SageMaker model
            create_model_response = sagemaker_client.create_model(
                ModelName=model_name,
                PrimaryContainer={
                    'Image': training_job['AlgorithmSpecification']['TrainingImage'],
                    'ModelDataUrl': model_data_url,
                    'Environment': {
                        'SAGEMAKER_PROGRAM': 'inference.py',
                        'SAGEMAKER_SUBMIT_DIRECTORY': model_data_url,
                        'SAGEMAKER_CONTAINER_LOG_LEVEL': '20',
                        'SAGEMAKER_REGION': PRIMARY_REGION
                    }
                },
                ExecutionRoleArn=role
            )
            print(f"✓ Model created: {model_name}")
            
            # Create endpoint configuration
            endpoint_config_name = f"{model_name}-config"
            print(f"Creating endpoint configuration: {endpoint_config_name}")
            
            endpoint_config_response = sagemaker_client.create_endpoint_config(
                EndpointConfigName=endpoint_config_name,
                ProductionVariants=[{
                    'InstanceType': 'ml.m5.xlarge',
                    'InitialInstanceCount': 1,
                    'ModelName': model_name,
                    'VariantName': 'AllTraffic'
                }]
            )
            print(f"✓ Endpoint configuration created")
            
            # Create endpoint
            endpoint_name = f"financial-qa-endpoint-{int(time.time())}"
            print(f"Creating endpoint: {endpoint_name}")
            print("⏳ This may take 5-10 minutes...")
            
            endpoint_response = sagemaker_client.create_endpoint(
                EndpointName=endpoint_name,
                EndpointConfigName=endpoint_config_name
            )
            
            # Wait for endpoint to be in service
            print("Waiting for endpoint to be in service...")
            waiter = sagemaker_client.get_waiter('endpoint_in_service')
            waiter.wait(EndpointName=endpoint_name)
            
            print(f"\n✓ Endpoint deployed successfully!")
            
            return {
                "endpoint_name": endpoint_name,
                "endpoint_config_name": endpoint_config_name,
                "model_name": model_name,
                "status": "InService"
            }
        else:
            print("⚠️  No training job name provided.")
            print("\nTo deploy a model:")
            print("1. Complete a SageMaker training job")
            print("2. Note the training job name")
            print("3. Run: deploy_fine_tuned_model('your-training-job-name')")
            
            print("\n📝 Deployment Process:")
            print("   • Creates SageMaker Model from training artifacts")
            print("   • Creates Endpoint Configuration with instance type")
            print("   • Deploys to real-time inference endpoint")
            print("   • Waits for endpoint to be InService (5-10 minutes)")
            
            return None
            
    except Exception as e:
        print(f"❌ Error deploying model: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# Example usage
print("="*80)
print("MODEL DEPLOYMENT")
print("="*80)
print("\n✓ Deployment function ready")
print("\nUsage:")
print("  endpoint_info = deploy_fine_tuned_model('your-training-job-name')")
print("\nOr call without parameters to see instructions:")
print("  deploy_fine_tuned_model()")

### Step 3: Test the Deployed Model

Invoke the endpoint to test the fine-tuned model's responses.

In [ ]:
def test_sagemaker_endpoint(endpoint_name, test_questions=None):
    """
    Test a deployed SageMaker endpoint with financial questions.
    
    Args:
        endpoint_name: Name of the deployed endpoint
        test_questions: List of test questions (optional)
        
    Returns:
        List of test results with predictions
    """
    if not test_questions:
        test_questions = [
            "What is a 401(k)?",
            "How does compound interest work?",
            "What is an ETF?",
            "What is a credit score?"
        ]
    
    print("="*80)
    print(f"TESTING ENDPOINT: {endpoint_name}")
    print("="*80)
    
    results = []
    
    for i, question in enumerate(test_questions, 1):
        print(f"\n[Test {i}/{len(test_questions)}] Question: {question}")
        
        try:
            # Prepare payload
            payload = {
                "inputs": f"Question: {question}\nAnswer:",
                "parameters": {
                    "max_new_tokens": 100,
                    "temperature": 0.7,
                    "top_p": 0.9
                }
            }
            
            # Invoke endpoint
            response = sagemaker_client.invoke_endpoint(
                EndpointName=endpoint_name,
                ContentType='application/json',
                Body=json.dumps(payload)
            )
            
            # Parse response
            result = json.loads(response['Body'].read().decode())
            
            # Extract generated text
            if isinstance(result, list) and len(result) > 0:
                answer = result[0].get('generated_text', 'No answer generated')
            elif isinstance(result, dict):
                answer = result.get('generated_text', result.get('predictions', 'No answer generated'))
            else:
                answer = str(result)
            
            print(f"✓ Answer: {answer[:200]}...")
            
            results.append({
                "question": question,
                "answer": answer,
                "success": True,
                "timestamp": datetime.now().isoformat()
            })
            
        except Exception as e:
            print(f"❌ Error: {str(e)}")
            results.append({
                "question": question,
                "answer": None,
                "success": False,
                "error": str(e),
                "timestamp": datetime.now().isoformat()
            })
    
    # Summary
    print("\n" + "="*80)
    print("TEST SUMMARY")
    print("="*80)
    successful = sum(1 for r in results if r['success'])
    print(f"Successful: {successful}/{len(results)}")
    print(f"Failed: {len(results) - successful}/{len(results)}")
    
    return results

# Example usage
print("✓ Endpoint testing function ready")
print("\nTo test an endpoint:")
print("  results = test_sagemaker_endpoint('your-endpoint-name')")
print("\nWith custom questions:")
print("  results = test_sagemaker_endpoint('endpoint-name', ['What is a dividend?', 'Explain mutual funds'])")

### Step 4: Implement Model Monitoring with CloudWatch

Set up CloudWatch monitoring to track model performance and detect drift.

In [ ]:
def setup_model_monitoring(endpoint_name):
    """
    Configure CloudWatch monitoring and alarms for a SageMaker endpoint.
    
    Args:
        endpoint_name: Name of the endpoint to monitor
        
    Returns:
        Dictionary with monitoring configuration details
    """
    try:
        print("="*80)
        print(f"SETTING UP MONITORING FOR: {endpoint_name}")
        print("="*80)
        
        # 1. Create CloudWatch Alarms
        alarms_created = []
        
        # High latency alarm
        print("\n[1/4] Creating high latency alarm...")
        alarm_name_latency = f"{endpoint_name}-high-latency"
        
        cloudwatch_client.put_metric_alarm(
            AlarmName=alarm_name_latency,
            ComparisonOperator='GreaterThanThreshold',
            EvaluationPeriods=2,
            MetricName='ModelLatency',
            Namespace='AWS/SageMaker',
            Period=300,  # 5 minutes
            Statistic='Average',
            Threshold=5000.0,  # 5 seconds
            ActionsEnabled=False,  # Set to True with SNS topic for notifications
            AlarmDescription=f'Alarm when model latency exceeds 5 seconds',
            Dimensions=[
                {'Name': 'EndpointName', 'Value': endpoint_name},
                {'Name': 'VariantName', 'Value': 'AllTraffic'}
            ]
        )
        print(f"✓ Created alarm: {alarm_name_latency}")
        alarms_created.append(alarm_name_latency)
        
        # High error rate alarm
        print("\n[2/4] Creating high error rate alarm...")
        alarm_name_errors = f"{endpoint_name}-high-errors"
        
        cloudwatch_client.put_metric_alarm(
            AlarmName=alarm_name_errors,
            ComparisonOperator='GreaterThanThreshold',
            EvaluationPeriods=1,
            MetricName='ModelInvocation4XXErrors',
            Namespace='AWS/SageMaker',
            Period=300,
            Statistic='Sum',
            Threshold=10.0,
            ActionsEnabled=False,
            AlarmDescription=f'Alarm when 4XX errors exceed 10 in 5 minutes',
            Dimensions=[
                {'Name': 'EndpointName', 'Value': endpoint_name},
                {'Name': 'VariantName', 'Value': 'AllTraffic'}
            ]
        )
        print(f"✓ Created alarm: {alarm_name_errors}")
        alarms_created.append(alarm_name_errors)
        
        # CPU utilization alarm
        print("\n[3/4] Creating CPU utilization alarm...")
        alarm_name_cpu = f"{endpoint_name}-high-cpu"
        
        cloudwatch_client.put_metric_alarm(
            AlarmName=alarm_name_cpu,
            ComparisonOperator='GreaterThanThreshold',
            EvaluationPeriods=2,
            MetricName='CPUUtilization',
            Namespace='/aws/sagemaker/Endpoints',
            Period=300,
            Statistic='Average',
            Threshold=80.0,  # 80%
            ActionsEnabled=False,
            AlarmDescription=f'Alarm when CPU utilization exceeds 80%',
            Dimensions=[
                {'Name': 'EndpointName', 'Value': endpoint_name},
                {'Name': 'VariantName', 'Value': 'AllTraffic'}
            ]
        )
        print(f"✓ Created alarm: {alarm_name_cpu}")
        alarms_created.append(alarm_name_cpu)
        
        # 2. Create CloudWatch Dashboard
        print("\n[4/4] Creating CloudWatch dashboard...")
        dashboard_name = f"{endpoint_name}-dashboard"
        
        dashboard_body = {
            "widgets": [
                {
                    "type": "metric",
                    "properties": {
                        "metrics": [
                            ["AWS/SageMaker", "ModelLatency", {"stat": "Average"}],
                            [".", ".", {"stat": "p99"}]
                        ],
                        "period": 300,
                        "stat": "Average",
                        "region": PRIMARY_REGION,
                        "title": "Model Latency",
                        "yAxis": {"left": {"label": "Milliseconds"}},
                        "dimensions": {
                            "EndpointName": endpoint_name,
                            "VariantName": "AllTraffic"
                        }
                    }
                },
                {
                    "type": "metric",
                    "properties": {
                        "metrics": [
                            ["AWS/SageMaker", "Invocations", {"stat": "Sum"}],
                            [".", "Invocation4XXErrors", {"stat": "Sum"}],
                            [".", "Invocation5XXErrors", {"stat": "Sum"}]
                        ],
                        "period": 300,
                        "stat": "Sum",
                        "region": PRIMARY_REGION,
                        "title": "Invocations and Errors",
                        "dimensions": {
                            "EndpointName": endpoint_name,
                            "VariantName": "AllTraffic"
                        }
                    }
                },
                {
                    "type": "metric",
                    "properties": {
                        "metrics": [
                            ["/aws/sagemaker/Endpoints", "CPUUtilization", {"stat": "Average"}],
                            [".", "MemoryUtilization", {"stat": "Average"}]
                        ],
                        "period": 300,
                        "stat": "Average",
                        "region": PRIMARY_REGION,
                        "title": "Resource Utilization",
                        "yAxis": {"left": {"label": "Percent"}},
                        "dimensions": {
                            "EndpointName": endpoint_name,
                            "VariantName": "AllTraffic"
                        }
                    }
                }
            ]
        }
        
        cloudwatch_client.put_dashboard(
            DashboardName=dashboard_name,
            DashboardBody=json.dumps(dashboard_body)
        )
        print(f"✓ Created dashboard: {dashboard_name}")
        
        print("\n" + "="*80)
        print("MONITORING SETUP COMPLETE")
        print("="*80)
        print(f"\n✓ Alarms created: {len(alarms_created)}")
        for alarm in alarms_created:
            print(f"   • {alarm}")
        
        print(f"\n✓ Dashboard: {dashboard_name}")
        print(f"   View at: https://console.aws.amazon.com/cloudwatch/home?region={PRIMARY_REGION}#dashboards:name={dashboard_name}")
        
        return {
            "alarms": alarms_created,
            "dashboard_name": dashboard_name,
            "dashboard_url": f"https://console.aws.amazon.com/cloudwatch/home?region={PRIMARY_REGION}#dashboards:name={dashboard_name}"
        }
        
    except Exception as e:
        print(f"❌ Error setting up monitoring: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# Example usage
print("✓ Model monitoring function ready")
print("\nTo set up monitoring:")
print("  monitoring_info = setup_model_monitoring('your-endpoint-name')")
print("\nMonitoring includes:")
print("  • High latency alarm (>5 seconds)")
print("  • High error rate alarm (>10 errors/5min)")
print("  • High CPU utilization alarm (>80%)")
print("  • CloudWatch dashboard with key metrics")

### Step 5: Model Lifecycle Management

Implement automated model versioning, A/B testing, and cleanup procedures.

In [ ]:
def implement_ab_testing(endpoint_name, new_model_name, traffic_split=0.1):
    """
    Implement A/B testing by updating endpoint with multiple model variants.
    
    Args:
        endpoint_name: Existing endpoint name
        new_model_name: Name of the new model to test
        traffic_split: Percentage of traffic for new model (0.0-1.0)
        
    Returns:
        Dictionary with updated endpoint configuration
    """
    try:
        print("="*80)
        print("IMPLEMENTING A/B TESTING")
        print("="*80)
        
        # Get current endpoint configuration
        print(f"\n[1/4] Getting current endpoint configuration...")
        endpoint_desc = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
        current_config = endpoint_desc['EndpointConfigName']
        print(f"✓ Current config: {current_config}")
        
        # Create new endpoint configuration with two variants
        print(f"\n[2/4] Creating new endpoint configuration with A/B split...")
        new_config_name = f"{endpoint_name}-ab-{int(time.time())}"
        
        # Calculate traffic distribution
        new_variant_weight = int(traffic_split * 100)
        current_variant_weight = 100 - new_variant_weight
        
        production_variants = [
            {
                'VariantName': 'VariantA',
                'ModelName': endpoint_desc['ProductionVariants'][0]['VariantName'],
                'InitialInstanceCount': 1,
                'InstanceType': 'ml.m5.xlarge',
                'InitialVariantWeight': current_variant_weight
            },
            {
                'VariantName': 'VariantB',
                'ModelName': new_model_name,
                'InitialInstanceCount': 1,
                'InstanceType': 'ml.m5.xlarge',
                'InitialVariantWeight': new_variant_weight
            }
        ]
        
        sagemaker_client.create_endpoint_config(
            EndpointConfigName=new_config_name,
            ProductionVariants=production_variants
        )
        print(f"✓ Created config: {new_config_name}")
        print(f"   • Variant A (current): {current_variant_weight}% traffic")
        print(f"   • Variant B (new): {new_variant_weight}% traffic")
        
        # Update endpoint
        print(f"\n[3/4] Updating endpoint...")
        print("⏳ This may take several minutes...")
        
        sagemaker_client.update_endpoint(
            EndpointName=endpoint_name,
            EndpointConfigName=new_config_name
        )
        
        # Wait for update to complete
        waiter = sagemaker_client.get_waiter('endpoint_in_service')
        waiter.wait(EndpointName=endpoint_name)
        
        print(f"✓ Endpoint updated successfully")
        
        # Set up variant-specific CloudWatch metrics
        print(f"\n[4/4] Setting up variant metrics...")
        
        for variant in ['VariantA', 'VariantB']:
            alarm_name = f"{endpoint_name}-{variant}-performance"
            
            cloudwatch_client.put_metric_alarm(
                AlarmName=alarm_name,
                ComparisonOperator='GreaterThanThreshold',
                EvaluationPeriods=2,
                MetricName='ModelLatency',
                Namespace='AWS/SageMaker',
                Period=300,
                Statistic='Average',
                Threshold=6000.0,
                ActionsEnabled=False,
                AlarmDescription=f'Performance monitoring for {variant}',
                Dimensions=[
                    {'Name': 'EndpointName', 'Value': endpoint_name},
                    {'Name': 'VariantName', 'Value': variant}
                ]
            )
        
        print(f"✓ Variant metrics configured")
        
        print("\n" + "="*80)
        print("A/B TESTING ACTIVE")
        print("="*80)
        print(f"\n✓ Endpoint: {endpoint_name}")
        print(f"✓ Traffic split: {current_variant_weight}% / {new_variant_weight}%")
        print("\n📊 Monitor metrics in CloudWatch to compare variants")
        print("📝 After sufficient data, promote the better variant to 100%")
        
        return {
            "endpoint_name": endpoint_name,
            "config_name": new_config_name,
            "variant_a_weight": current_variant_weight,
            "variant_b_weight": new_variant_weight,
            "status": "Active"
        }
        
    except Exception as e:
        print(f"❌ Error implementing A/B testing: {str(e)}")
        import traceback
        traceback.print_exc()
        return None


def cleanup_old_resources(days_old=7, dry_run=True):
    """
    Clean up old SageMaker resources to manage costs.
    
    Args:
        days_old: Delete resources older than this many days
        dry_run: If True, only list resources without deleting
        
    Returns:
        Dictionary with cleanup summary
    """
    try:
        print("="*80)
        print(f"RESOURCE CLEANUP {'(DRY RUN)' if dry_run else ''}")
        print("="*80)
        
        from datetime import timedelta
        cutoff_date = datetime.now() - timedelta(days=days_old)
        
        cleanup_summary = {
            "endpoints": [],
            "endpoint_configs": [],
            "models": []
        }
        
        # Find old endpoints
        print(f"\n[1/3] Checking endpoints (older than {days_old} days)...")
        endpoints = sagemaker_client.list_endpoints()
        
        for endpoint in endpoints.get('Endpoints', []):
            endpoint_name = endpoint['EndpointName']
            creation_time = endpoint['CreationTime'].replace(tzinfo=None)
            
            if creation_time < cutoff_date:
                print(f"   • {endpoint_name} (created {creation_time.date()})")
                cleanup_summary["endpoints"].append(endpoint_name)
                
                if not dry_run:
                    sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
                    print(f"     ✓ Deleted")
        
        # Find old endpoint configs
        print(f"\n[2/3] Checking endpoint configurations...")
        configs = sagemaker_client.list_endpoint_configs()
        
        for config in configs.get('EndpointConfigs', []):
            config_name = config['EndpointConfigName']
            creation_time = config['CreationTime'].replace(tzinfo=None)
            
            if creation_time < cutoff_date:
                print(f"   • {config_name} (created {creation_time.date()})")
                cleanup_summary["endpoint_configs"].append(config_name)
                
                if not dry_run:
                    sagemaker_client.delete_endpoint_config(EndpointConfigName=config_name)
                    print(f"     ✓ Deleted")
        
        # Find old models
        print(f"\n[3/3] Checking models...")
        models = sagemaker_client.list_models()
        
        for model in models.get('Models', []):
            model_name = model['ModelName']
            creation_time = model['CreationTime'].replace(tzinfo=None)
            
            if creation_time < cutoff_date:
                print(f"   • {model_name} (created {creation_time.date()})")
                cleanup_summary["models"].append(model_name)
                
                if not dry_run:
                    sagemaker_client.delete_model(ModelName=model_name)
                    print(f"     ✓ Deleted")
        
        print("\n" + "="*80)
        print("CLEANUP SUMMARY")
        print("="*80)
        print(f"Endpoints: {len(cleanup_summary['endpoints'])}")
        print(f"Endpoint Configs: {len(cleanup_summary['endpoint_configs'])}")
        print(f"Models: {len(cleanup_summary['models'])}")
        
        if dry_run:
            print("\n⚠️  DRY RUN: No resources were deleted")
            print("   Set dry_run=False to actually delete resources")
        else:
            print("\n✓ Resources deleted successfully")
        
        return cleanup_summary
        
    except Exception as e:
        print(f"❌ Error during cleanup: {str(e)}")
        import traceback
        traceback.print_exc()
        return None


# Example usage
print("✓ Model lifecycle management functions ready")
print("\n1. A/B Testing:")
print("   ab_info = implement_ab_testing('endpoint-name', 'new-model-name', traffic_split=0.1)")
print("\n2. Resource Cleanup (dry run):")
print("   summary = cleanup_old_resources(days_old=7, dry_run=True)")
print("\n3. Resource Cleanup (execute):")
print("   summary = cleanup_old_resources(days_old=7, dry_run=False)")

## Summary: Complete AI Assistant Implementation

This notebook has demonstrated a production-ready AI Assistant with:

### ✅ Part 1: Foundation Model Benchmarking
- Evaluated Claude and Titan models on financial domain tasks
- Measured latency, quality (similarity scores), and cost
- Created data-driven model selection strategy

### ✅ Part 2: Flexible Architecture
- AWS AppConfig for dynamic model routing
- Lambda-based model abstraction layer
- API Gateway integration for external access

### ✅ Part 3: Resilient System Design
- Step Functions circuit breaker pattern
- Automatic fallback to secondary models
- Graceful degradation with regulation-compliant responses
- CloudFormation templates for cross-region deployment
- Route 53 DNS failover for high availability

### ✅ Part 4: Model Customization & Lifecycle
- SageMaker fine-tuning with HuggingFace
- Model deployment to real-time endpoints
- CloudWatch monitoring and alerting
- A/B testing for model comparison
- Automated resource cleanup

### 🎯 Key Features Implemented:
1. **Multi-model support** with automatic selection
2. **Fault tolerance** with circuit breakers and fallbacks
3. **High availability** with cross-region deployment
4. **Monitoring & alerting** with CloudWatch
5. **Model versioning** and A/B testing
6. **Cost optimization** with automated cleanup

### 📚 Next Steps:
- Request GPU quota for actual model fine-tuning
- Deploy to production with custom domain
- Configure SNS notifications for alarms
- Implement model drift detection
- Add authentication/authorization (API keys, Cognito)
- Set up CI/CD pipeline for automated deployments